In [1]:
import json
import datetime
import requests
import pandas as pd
import numpy as np
import re

In [2]:
from datetime import datetime, timedelta

In [3]:
# for fetching team divisons and conferences 
from nba_api.stats.endpoints._base import Endpoint
from nba_api.stats.library.http import NBAStatsHTTP
from nba_api.stats.library.parameters import (
    LeagueID,
    Season,
    SeasonType,
    SeasonNullable,
)
from nba_api.stats.endpoints.leaguestandingsv3 import LeagueStandingsV3

# Import Schedule

In [4]:
year='2024'
r = requests.get('https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/' + year + '/league/00_full_schedule.json')
json_data = r.json()

In [5]:
data = [
    {
        'Game ID': game['gid'],
        'Game Code': game['gcode'],
        'Game Date': game['gdte'],
        'Arena': game['an'],
        'Arena City': game['ac'],
        'Arena State': game['as'],
        'Home Team Abbreviation': game['h']['ta'],
        'Home Team Name': game['h']['tn'],
        'Home Team City': game['h']['tc'],
        'Home Team Score': game['h']['s'],
        'Visiting Team Abbreviation': game['v']['ta'],
        'Visiting Team Name': game['v']['tn'],
        'Visiting Team City': game['v']['tc'],
        'Visiting Team Score': game['v']['s'],
        'Game Time': game['stt'],
        'Game Status': game['st'],
        'Game Date UTC': game['gdtutc'],
        'UTC Time': game['utctm']
    }
    for month_schedule in json_data['lscd']
    for game in month_schedule['mscd']['g']
]

# Create DataFrame directly from the list of dictionaries
df = pd.DataFrame(data)

In [6]:
# Sort the DataFrame by the "Game Date" column
schedule_df = df.sort_values(by="Game Date")
schedule_df_clean = schedule_df.drop(columns=["Game Date UTC", "UTC Time", "Game Status", "Visiting Team City", "Home Team City", "Game Code", "Arena State"])
# If you want to reset the index after sorting
schedule = schedule_df_clean.reset_index(drop=True).copy()

# Functions 

In [7]:
def get_todays_games(df, today):
    todays_games=df[df["Game Date"] == today]
    return todays_games

In [8]:
get_todays_games(schedule, '2024-10-23')

,Game ID,Game Date,Arena,Arena City,Home Team Abbreviation,Home Team Name,Home Team Score,Visiting Team Abbreviation,Visiting Team Name,Visiting Team Score,Game Time
2,0022400067,2024-10-23,Scotiabank Arena,Toronto,TOR,Raptors,,CLE,Cavaliers,,7:30 pm ET
3,0022400072,2024-10-23,Moda Center,Portland,POR,Trail Blazers,,GSW,Warriors,,10:00 pm ET
4,0022400071,2024-10-23,Intuit Dome,Inglewood,LAC,Clippers,,PHX,Suns,,10:00 pm ET
5,0022400070,2024-10-23,Delta Center,Salt Lake City,UTA,Jazz,,MEM,Grizzlies,,9:00 pm ET
6,0022400066,2024-10-23,Wells Fargo Center,Philadelphia,PHI,76ers,,MIL,Bucks,,7:30 pm ET
7,0022400068,2024-10-23,Toyota Center,Houston,HOU,Rockets,,CHA,Hornets,,8:00 pm ET
8,0022400065,2024-10-23,Kaseya Center,Miami,MIA,Heat,,ORL,Magic,,7:30 pm ET
9,0022400064,2024-10-23,State Farm Arena,Atlanta,ATL,Hawks,,BKN,Nets,,7:30 pm ET
10,0022400069,2024-10-23,Smoothie King Center,New Orleans,NOP,Pelicans,,CHI,Bulls,,8:00 pm ET
11,0022400063,2024-10-23,Little Caesars Arena,Detroit,DET,Pistons,,IND,Pacers,,7:00 pm ET


# Get Teams and Divisions

In [82]:
from nba_api.stats.static import teams

# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = teams.get_teams()
print("Number of teams fetched: {}".format(len(nba_teams)))
nba_teams[0:29]
teams = pd.DataFrame(nba_teams)
teams = teams.set_index('id') 

Number of teams fetched: 30


In [83]:
def season_standings(season, season_type): 
    league_standings = LeagueStandingsV3(
        league_id="00",
        season=season,              # Example: "2022-23" for the 2022-2023 season
        season_type=season_type   # Example: "Regular Season", "Playoffs", etc.
    )
    standings_data = league_standings.standings.get_dict()
    standings = pd.DataFrame(standings_data['data'], columns=standings_data['headers'])
    standings.index = np.arange(1, len(standings) + 1)
    standings = standings.drop(columns=['SeasonID','LeagueID', 'TeamSlug', 'LeagueRank','LongHomeStreak', 'LongRoadStreak','CurrentHomeStreak','CurrentRoadStreak','CurrentStreak'])
    return standings 

In [84]:
league_standings = season_standings('2023-24', 'Regular Season')

In [85]:
league_standings = league_standings.rename(columns={'TeamID': 'id'})

# Step 2: Merge the DataFrames on the 'id' column
teams_divisions = pd.merge(teams, league_standings[['id', 'Division', 'Conference']], on='id', how='left')


In [86]:
# Merge for Home Teams
schedule1 = schedule.merge(teams_divisions[['abbreviation', 'Division', 'Conference']], 
                          left_on='Home Team Abbreviation', right_on='abbreviation', 
                          how='left').rename(columns={'Division': 'Home Division', 'Conference': 'Home Conference'})

# Merge for Visiting Teams
schedule2 = schedule1.merge(teams_divisions[['abbreviation', 'Division', 'Conference']], 
                          left_on='Visiting Team Abbreviation', right_on='abbreviation', 
                          how='left').rename(columns={'Division': 'Visiting Division', 'Conference': 'Visiting Conference'})

In [111]:
# Divisional Game: Same division for both teams
schedule2['Divisional Game'] = schedule2['Home Division'] == schedule2['Visiting Division']

# Conference Game: Same conference for both teams
schedule2['Conference Game'] = schedule2['Home Conference'] == schedule2['Visiting Conference']

# Step 3: Convert the boolean values to more descriptive text if desired
schedule2['Divisional Game'] = schedule2['Divisional Game'].apply(lambda x: 'Yes' if x else 'No')
schedule2['Conference Game'] = schedule2['Conference Game'].apply(lambda x: 'Yes' if x else 'No')

In [112]:
final_schedule = schedule2.drop(columns=['Visiting Team Score', 'Home Team Score', "abbreviation_x", "abbreviation_y"])

In [113]:
final_schedule = final_schedule.drop(columns=["Home Team Name", "Visiting Team Name"])

In [115]:
new_column_order = ['Game Date', 'Game ID', 'Game Time', 'Arena', 'Arena City', 'Home Team Abbreviation', 'Home Conference', 'Home Division', 'Visiting Team Abbreviation', 'Visiting Conference', 'Visiting Division', 'Divisional Game', 'Conference Game'] 

In [116]:
nbaSchedule24 = final_schedule[new_column_order]

In [118]:
nbaSchedule24.to_csv('nbaSchedule24.csv', index=False)